In [5]:
%pip install scikit-learn

  Obtaining dependency information for scikit-learn from https://files.pythonhosted.org/packages/37/a9/08538c29f188b1c37eb6e115a5046c6d17457e1a7b8ea748edd3017ddd6c/scikit_learn-1.3.1-cp312-cp312-win_amd64.whl.metadata
  Obtaining dependency information for scipy>=1.5.0 from https://files.pythonhosted.org/packages/f4/ce/be0b376ba6069f3f8ba240aa532a374733447453c93582d4c474effdde21/scipy-1.11.3-cp312-cp312-win_amd64.whl.metadata
     ---------------------------------------- 0.0/60.4 kB ? eta -:--:--
     --------------------------- ------------ 41.0/60.4 kB 1.9 MB/s eta 0:00:01
     ---------------------------------------- 60.4/60.4 kB 1.6 MB/s eta 0:00:00
  Obtaining dependency information for joblib>=1.1.1 from https://files.pythonhosted.org/packages/10/40/d551139c85db202f1f384ba8bcf96aca2f329440a844f924c8a0040b6d02/joblib-1.3.2-py3-none-any.whl.metadata
  Using cached joblib-1.3.2-py3-none-any.whl.metadata (5.4 kB)
  Obtaining dependency information for threadpoolctl>=2.0.0 from https:

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression

In [48]:
ntrain = 50 # Training
ntest = 500 # Test
nrep = 100 # repeat 100 times
p = 20
puse = np.array([1, 2, 3, 4, 10, 20]) # number of predictors
k = np.arange(1 ,10)

sigma = 0.5

Xtrain = np.random.uniform(-1, 1, [ntrain,p])
Xtest = np.random.uniform(-1, 1, [ntest,p])
y0 = np.sin(2 * Xtrain[:, 0]) # Only the first predictor is related to y
ytest = np.sin(2 * Xtest[:, 0])

out_knn = pd.DataFrame() # Output results
out_lm = pd.DataFrame()


In [49]:
for i in np.arange(0, len(puse)):
    yhat_lm = np.zeros((ntrain, nrep))
    yhat_knn = np.zeros((ntrain, nrep, len(k)))
    
    for l in np.arange(0, nrep):
        y = y0 + np.random.normal(0, sigma, ntrain)
    
        ######### DO: fit linear regression using LinearRegression funciton, assign predicted value to yhat_lm[:, [l]] #########
        fit_lm = LinearRegression().fit(Xtrain[:, 0:puse[i]], y)
        yhat_lm[:, l] = fit_lm.predict(Xtrain[:, 0:puse[i]]) # Predicted value
      
    
        for j in np.arange(0, len(k)):
            ######### DO: fit knn using KNeighborsRegressor funciton, assign predicted value to yhat_knn[:, l, j] #########
            fit_knn = KNeighborsRegressor(j+1).fit(Xtrain[:, 0:puse[i]], y)
            yhat_knn[:, l, j] = fit_knn.predict(Xtrain[:, 0:puse[i]])  # Predicted value
        
        
        print(i + 1, "-th p, ", l + 1, "-th repetition finished. \n")


    ######### DO: compute bias and variance of linear regression #########
    
    # Compute mean of predicted values
    ybar_lm = np.mean(yhat_lm, axis=1) # E(f^hat)
    
    # Compute bias^2
    ylm = np.average(ytest.reshape(-1, 10), axis=1) 
    biasSQ_lm = np.mean(np.square((ylm - ybar_lm))) # E[ (f - E(f^hat))^2 ]
    
    # Compute variance
    ybar_bar = np.mean(ybar_lm)
    variance_lm = np.mean(np.square(ybar_lm-ybar_bar)) # E[ (E(f^hat) - f^hat)^2 ]
    
    # Compute total MSE
    err_lm = np.sum(np.square(ybar_lm-ylm))
  
    out_lm = pd.concat((out_lm, pd.DataFrame({'error': [biasSQ_lm], 'component': ['squared-bias'], 'p': [f"p = {puse[i]}"]})))
    out_lm = pd.concat((out_lm, pd.DataFrame({'error': [variance_lm], 'component': ['variance'], 'p': [f"p = {puse[i]}"]})))
    out_lm = pd.concat((out_lm, pd.DataFrame({'error': [err_lm], 'component': ['MSE'], 'p': [f"p = {puse[i]}"]})))
    ######### DO: compute bias and variance of knn regression #########
    
    # Compute mean of predicted values
    ybar_knn = np.mean(np.mean(yhat_knn, axis=2), axis=1) # E(f^hat)
    
    # Compute bias^2
    yknn = np.average(ytest.reshape(-1, 10), axis=1) 
    biasSQ_knn = np.mean(np.square((yknn - ybar_knn))) # E[ (f - E(f^hat))^2 ]
    
    # Compute variance
    yknn_bar_bar = np.mean(ybar_knn)
    variance_knn = np.mean(np.square(ybar_knn-yknn_bar_bar)) # E[ (E(f^hat) - f^hat)^2 ]
    
    # Compute total MSE
    err_knn = np.sum(np.square(ybar_knn-yknn))
    
    out_knn = pd.concat((out_knn, pd.DataFrame({'error': biasSQ_knn, 'component': ['squared-bias'] * len(k), 'K': 1 / np.array(k),
                                       'p': [f"p = {puse[i]}"] * len(k)})))
    out_knn = pd.concat((out_knn, pd.DataFrame({'error': variance_knn, 'component': ['variance'] * len(k), 'K': 1 / np.array(k),
                                       'p': [f"p = {puse[i]}"] * len(k)})))
    out_knn = pd.concat((out_knn, pd.DataFrame({'error': err_knn, 'component': ['MSE'] * len(k), 'K': 1 / np.array(k),
                                       'p': [f"p = {puse[i]}"] * len(k)})))

1 -th p,  1 -th repetition finished. 

1 -th p,  2 -th repetition finished. 

1 -th p,  3 -th repetition finished. 

1 -th p,  4 -th repetition finished. 

1 -th p,  5 -th repetition finished. 

1 -th p,  6 -th repetition finished. 

1 -th p,  7 -th repetition finished. 

1 -th p,  8 -th repetition finished. 

1 -th p,  9 -th repetition finished. 

1 -th p,  10 -th repetition finished. 

1 -th p,  11 -th repetition finished. 

1 -th p,  12 -th repetition finished. 

1 -th p,  13 -th repetition finished. 

1 -th p,  14 -th repetition finished. 

1 -th p,  15 -th repetition finished. 

1 -th p,  16 -th repetition finished. 

1 -th p,  17 -th repetition finished. 

1 -th p,  18 -th repetition finished. 

1 -th p,  19 -th repetition finished. 

1 -th p,  20 -th repetition finished. 

1 -th p,  21 -th repetition finished. 

1 -th p,  22 -th repetition finished. 

1 -th p,  23 -th repetition finished. 

1 -th p,  24 -th repetition finished. 

1 -th p,  25 -th repetition finished. 

1 -th p, 

In [50]:
out_lm

,error,component,p
0,0.545624,squared-bias,p = 1
0,0.549018,variance,p = 1
0,27.281208,MSE,p = 1
0,0.534673,squared-bias,p = 2
0,0.538048,variance,p = 2
0,26.733674,MSE,p = 2
0,0.530091,squared-bias,p = 3
0,0.530597,variance,p = 3
0,26.504558,MSE,p = 3
0,0.524459,squared-bias,p = 4


In [54]:
%pip install tabulate

  Using cached tabulate-0.9.0-py3-none-any.whl (35 kB)
Note: you may need to restart the kernel to use updated packages.


In [59]:
with pd.option_context('display.max_rows', None,):
   print(out_knn)

       error     component         K       p
0   0.573112  squared-bias  1.000000   p = 1
1   0.573112  squared-bias  0.500000   p = 1
2   0.573112  squared-bias  0.333333   p = 1
3   0.573112  squared-bias  0.250000   p = 1
4   0.573112  squared-bias  0.200000   p = 1
5   0.573112  squared-bias  0.166667   p = 1
6   0.573112  squared-bias  0.142857   p = 1
7   0.573112  squared-bias  0.125000   p = 1
8   0.573112  squared-bias  0.111111   p = 1
0   0.565566      variance  1.000000   p = 1
1   0.565566      variance  0.500000   p = 1
2   0.565566      variance  0.333333   p = 1
3   0.565566      variance  0.250000   p = 1
4   0.565566      variance  0.200000   p = 1
5   0.565566      variance  0.166667   p = 1
6   0.565566      variance  0.142857   p = 1
7   0.565566      variance  0.125000   p = 1
8   0.565566      variance  0.111111   p = 1
0  28.655604           MSE  1.000000   p = 1
1  28.655604           MSE  0.500000   p = 1
2  28.655604           MSE  0.333333   p = 1
3  28.6556